In [1]:
import os
import tqdm
from pysat.pb import PBEnc
from pysat.formula import CNF
from pysat.solvers import Solver

In [32]:
for i in [6, 8, 10]:
    for j in [2]:
        s = ['c' + ' 0 1' * int(i / 2)]
        DATA_p = 'PPH/' + str(i) + '/' + str(j) + '/'
        DATA_f = os.listdir(DATA_p)
        cnf_t = CNF('PPH/' + str(i) + '/' + str(j) + '/ter.txt').clauses
        for f in DATA_f:
            if f != 'ter.txt':
                cnf_f = CNF(DATA_p + f)
                if cnf_f.comments == s:
                    cnf = atmost_for_each(i, j, 2) + cnf_t + cnf_f.clauses
                    solver = Solver('m22', cnf)
                    print(i, solver.solve())
#                     solution = filter(lambda x: abs(x) <= int(i * (i + 1) / 2) * j, solver.get_model())
#                     model = list(map(lambda x: toOp(x, i), solution))
#                     for order in toFull(model, i):
#                         print(order)
#                     for op in map(lambda x: [x[0], toOut(x[1]) + ' < ' + toOut(x[2])], model):
#                         print(op)
    #                     os.remove(DATA_p + f)
                    solver.delete()

6 False
8 False
10 False


In [ ]:
# 6 True
# c < a6 < a4 < a5 < a2 < a1 < a3
# a2 < a1 < a3 < c < a6 < a5 < a4
# 8 True
# c < a8 < a6 < a5 < a3 < a2 < a4 < a7 < a1
# a8 < a7 < a1 < c < a6 < a5 < a3 < a2 < a4
# 10 True
# c < a4 < a5 < a8 < a10 < a9 < a3 < a2 < a6 < a7 < a1
# a8 < a9 < a7 < c < a4 < a1 < a10 < a6 < a5 < a3 < a2

In [2]:
def toFull(ops, n):
    p = max(ops, key=lambda x: x[0])[0] + 1
    if len(ops) != p * int((n + 1) * n / 2):
        raise BaseException('Wrong len(ops)')
    res = []
    for i in range(p):
        ans = []
        for j in range(n + 1):
            best = [0] * (n + 1)
            for op in ops:
                if op[0] == i:
                    best[op[1]] += 1
            ans.append(best.index(j))
        ans.reverse()
        res.append(' < '.join(map(toOut, ans)))
    return(res)

In [8]:
os.makedirs('PPH')
for i in range(3, 21):
    for j in range(i + 1):
        os.makedirs('PPH' + '/' + str(i) + '/' + str(j))

In [4]:
def toVar(player, a, b, n):
    return int(player * (n + 1) * n / 2 + a * (2 * n - a - 1) / 2 + b)

def toLit(player, a, b, n):
    if a < b:
        return toVar(player, a, b, n)
    else:
        return -toVar(player, b, a, n)

def atmost_for_each(g_size, dif, k, var = 0):
    ans = []
    if var == 0:
        var = int((dif * (g_size + 1) * g_size) / 2)
    for i in range(dif):
        lits = []
        for j in range(1, g_size + 1):
            lits.append(toLit(i, j, 0, g_size))
        ans += PBEnc.atmost(lits, bound = k, top_id = var).clauses
        for clause in ans:
            for lit in clause:
                var = max(abs(lit), var)
    return ans

def atmost_for_all(g_size, dif, k, var = 0):
    lits = []
    if var == 0:
        var = int((dif * (g_size + 1) * g_size) / 2)
    for i in range(dif):
        for j in range(1, g_size + 1):
            lits.append(toLit(i, j, 0, g_size))
    return PBEnc.atmost(lits, bound = k, top_id = var).clauses

def atmost_combo(g_size, dif, k1, k2):
    var = 0
    cnf = atmost_for_each(g_size, dif, k1)
    for clause in cnf:
        for lit in clause:
            var = max(abs(lit), var)
    cnf += atmost_for_all(g_size, dif, k2, var)
    return cnf

def not_worst_for_one(g_size, dif, n):
    ans = []
    for i in range(dif):
        if i != n:
            for j in range(1, g_size + 1):
                ans.append([toLit(i, 0, j, g_size)])
    return ans

def c_worst_for(g_size, n):
    ans = []
    for i in range(1, g_size + 1):
        ans.append([toLit(n, 0, i, g_size)])
    return ans

def toOp(Lit, n):
    ans = [0] * 3
    aLit = abs(Lit) - 1
    num = int((n + 1) * n / 2)
    ans[0] = int(aLit / num)
    ab = aLit % num
    s = 0
    for i in range(n, 0, -1):
        s += i
        if ab < s:
            ans[1] = n - i
            ans[2] = n + ab - s + 1
            break
    if Lit < 0:
        ans[1], ans[2] = ans[2], ans[1]
    return(ans)

def toOut(num):
    return 'c' if num == 0 else 'a' + str(num)

In [ ]:
all_data = []
for i in range(2, 7):
    all_data.append([])
    for j in range(1, i + 1):
        all_data[-1].append([])
        DATA_p = 'data/' + str(i) + '/' + str(j) + '/'
        DATA_f = os.listdir(DATA_p)
        for f in tqdm.tqdm(DATA_f):
            cnf = CNF(DATA_p + f).clauses
            all_data[-1][-1].append(cnf)

In [ ]:
for i in range(2, 7):
    for j in range(1, i + 1):
        count = 0
        DATA_p = 'data/' + str(i) + '/' + str(j) + '/'
        DATA_f = os.listdir(DATA_p)
        cnf_t = CNF('CNF/' + str(i) + '/' + str(j) + '/ter/op.txt').clauses
        for f in tqdm.tqdm(DATA_f):
            cnf = atmost_for_all(i, j, 4) + cnf_t + CNF(DATA_p + f).clauses
            solver = Solver('m22', cnf)
            if solver.solve():
                count += 1
            solver.delete()
        print(i, j, count)

In [ ]:
for i in range(2, 7):
    for j in range(1, i + 1):
        count = 0
        cnf_t = CNF('CNF/' + str(i) + '/' + str(j) + '/ter/op.txt').clauses
        for cnf_f in tqdm.tqdm(all_data[i - 2][j - 1]):
            cnf = atmost_combo(i, j, 2, 4) + cnf_t + cnf_f
            solver = Solver('m22', cnf)
            if solver.solve():
                count += 1
            solver.delete()
        print(i, j, count)

In [ ]:
for i in range(2, 7):
    for j in range(1, i + 1):
        count = 0
        cnf_t = CNF('CNF/' + str(i) + '/' + str(j) + '/ter/op.txt').clauses
        for cnf_f in tqdm.tqdm(all_data[i - 2][j - 1]):
            for k in range(j):
                cnf = not_worst_for_one(i, j, k) + cnf_t + cnf_f
                solver = Solver('m22', cnf)
                if solver.solve():
                    count += 1
                solver.delete()
        print(i, j, count)

In [ ]:
Хотя бы 2 исхода должны быть хуже цикла для какого-то игрока. Хотя бы 4 исхода должны быть хуже цикла для всех игроков в сумме. Есть пример, где (1) и (2) выполняются одновременно. Нет примера, где цикл не худший исход ровно для одного игрока.